In [71]:
from pathlib import Path
from omnibelt import toposort, load_json, save_json, pformat, pformat_vars
from itertools import product, combinations
# import pyperclip
import random
from PIL import Image
from graphviz import Digraph
import matplotlib.pyplot as plt
import io
# import clipboard
import networkx as nx
from tqdm.notebook import tqdm
from tabulate import tabulate
import matplotlib.pyplot as plt
import seaborn as sns
from omniply import tool, ToolKit, Context

import numpy as np
import torch
from causalbenchmark.novo.simulation.models import Bernoulli, ConditionalBernoulli, BernoulliNetwork
from causalbenchmark.novo.seeding import NewsHeadlines, StatisticsPrompting, Story
# from causalbenchmark.novo.seeding.prompting import GraphInfo
from causalbenchmark.novo.templating import FixedTemplate, SimpleTemplater
# from causalbenchmark.novo.verbalization import misc
from causalbenchmark.novo import misc
from causalbenchmark.novo.verbalization.decision import Decision
from causalbenchmark.novo.templating import Template
from causalbenchmark.novo.verbalization import Verbalizer, MarginalVerbalization, ConditionalVerbalization
from causalbenchmark.novo.verbalization.variable import VariableVerbalization
from causalbenchmark.novo.verbalization.flavors import PrecisePercent
from causalbenchmark.novo.seeding.prompting import VerbalizationPrompting
from causalbenchmark.novo.simulation.solvers import ATE_Sign

In [72]:
path = misc.data_root() / 'stories'
story_names = [p.stem for p in path.glob('*.json')]#[:1]
print(story_names)

["Larisa Kuklina's Performance and Injury Analysis", 'Analyzing_the_Impact_of_Moon_New_Deal_Fund', 'Impact_of_SCGP_Deltalab_Acquisition_on_Revenue_Trends', 'Impact_of_Covid_on_Auto_Industry_Job_Market', 'Electra_SolarEnergyVenture_ImpactAnalysis', 'BelGee_Geely_Car_Production_Analysis', 'Impact_of_FDA_Menthol_Ban_on_Cigarette_Sales_and_Public_Health', 'Impact_of_Vrbetice_Compensation_on_Local_Economies_and_Living_Standards', 'Trend_Analysis_of_Retail_Sector_Lawsuits', 'Effectiveness_of_HPV_Self-Testing_in_Māori_Pasifika_Asian_Communities', 'Penguin_Populations_and_Climate_Change_Analysis', 'E-Bike Summer Sale Impact Analysis', 'NHS_Glasgow_Coronavirus_Outbreak_Impact_Study', 'Pandemic_Shopping_Trends_and_Parcel_Return_Rates_Analysis', 'Evaluating_Gogoro_Smart_Tram_Speed_Assist_Effectiveness', 'Impact_of_Child_Benefit_Payment_Timings_on_Household_Budgets_in_Greece', 'Oil_Price_Trends_and_Global_Economic_Indicators_Analysis', 'Impact_of_Benelux_Tour_on_Tour_of_Britain_Performance', 'Mete

In [73]:
use_old_loading = False
# story_names = [
# 	# 'old/test19', 'old/test20', 'old/test21',
# 	# 'Facebook-WallStreet_Trends_Post-Verdict1',
# 	# 'RoyalFashionTrends_Analysis2',
# 	# 'EU Gas Alliance and Inflation Trends',
# 	
# 	'Impact_of_Covid_on_Auto_Industry_Job_Market',
# 	
# ]
stories = []
for i, name in enumerate(story_names):
	story = Story(story_root=misc.data_root() / 'stories', story_id=name)#.populate_defaults()
	story['title'] = story['motivation']['title']
	story['index'] = i
	stories.append(story)
	if use_old_loading:
		story['stats'] = {int(k): v for k, v in story['stats'].items()}
		story['verbs'] = {k: {int(k2): v2 for k2, v2 in v.items()} for k, v in story['verbs'].items()}
		for k, v in story['verbs'].items():
			for k2, v2 in v.items():
				if 'value' in v2:
					v2['value'] = {int(k3): v3 for k3, v3 in v2['value'].items()}
		story['questions']['questions'] = {int(k): v for k, v in story['questions']['questions'].items()}
len(stories)

In [16]:
def old_story_atoms(story):
	varverbs = {}
	for var, raw in story['verbs'].items():
		info = {'values': {0: {}, 1: {}}}
		
		info['variable'] = raw[1]['variable']
		info['subject'] = raw[2]['subject']
		info['domain'] = raw[4]['domain']
		
		for i in [0, 1]:
			info_val = info['values'][i]
			info_val['verb'] = raw[2]['value'][i] # goes with subject
			info_val['phrase'] = raw[3]['value'][i]
			info_val['descriptor'] = raw[4]['value'][i] # goes with domain
			info_val['event'] = raw[5]['value'][i]
			info_val['conditional'] = raw[6]['value'][i]
			info_val['interventional'] = raw[7]['value'][i]
			
		varverbs[var] = info
	
	story['sheet'] = varverbs
	return story

def story_atoms(story):
	base_keys = ['descriptor', 'subject', 'pronoun', 'preposition', 'domain']
	value_keys = ['predicate', 'nounclause', 'subclause', 'condition', 'action']
	verbs = []
	for node in story['nodes']:
		try:
			raw = story['verbs'][node['name']]
		except:
			print(story['motivation']['title'])
			raise
		info = {'values': {0: {}, 1: {}}}
		info.update({key: raw[key] for key in base_keys})
		for i in [0, 1]:
			info_val = info['values'][i]
			info_val.update({key: raw[f'{key}{i}'] for key in value_keys})
		# node['verbs'] = info
		verbs.append(info)
	story['verbs'] = verbs
for story in stories:
	(old_story_atoms if use_old_loading else story_atoms)(story)

In [74]:
story = random.choice(stories)
# story = stories[4]
# rawverbs = story['verbs']
# rawvars = {node['name']: node for node in story['nodes']}
# next(iter(rawverbs.values()))

In [75]:
story['queries'][0]

{'treatment': 'Increase in Online Shopping',
 'query': 'ate',
 'type': 'ate-sign',
 'criterion': '<'}

In [80]:
qtype = 'ate-sign'
# qtype = 'cate-compare-c'

questions = []
for story in stories:
	qs = [{**question, **query, 'title': story['title'], 'story_index': story['index']} 
		  for query, question in zip(story['queries'], story['questions']) if query['type'] == qtype]
	questions.extend(qs)
len(questions)

In [81]:
print(VerbalizationPrompting._query_description_templates[qtype])

ATE({treatment!r}) {criterion} 0


In [84]:
questions[0]

{'question': "Does experiencing a hand injury typically lead to a decline in an athlete's performance?",
 'answer': 'yes',
 'explanation': "An athlete's performance generally decreases following a hand injury.",
 'treatment': 'Hand Injury',
 'query': 'ate',
 'type': 'ate-sign',
 'criterion': '>',
 'title': "Larisa Kuklina's Performance and Injury Analysis",
 'story_index': 0}

In [83]:
print(tabulate([[q['story_index'], q['question']] for q in questions]))

--  ---------------------------------------------------------------------------------------------------------------------------------------
 0  Does experiencing a hand injury typically lead to a decline in an athlete's performance?
 0  Is an athlete likely to experience a decline in performance when under mental stress?
 1  Does political stability generally lead to better outcomes for the country's economy?
 1  Does implementing the New Deal Fund negatively impact the country's economy?
 2  Does a higher level of market competition generally lead to a decrease in SCGP's yearly revenue growth?
 2  Is SCGP's acquisition of Deltalab likely to decrease their yearly revenue growth?
 3  Did the COVID-19 pandemic generally have a noticeable impact on the auto industry's job market?
 3  Did changes in government economic policies during the pandemic generally lead to noticeable effects in the auto industry's job market?
 4  Did Electra's acquisition of a solar panel company in Israel contrib

In [ ]:
templates = [
	''
]

In [67]:
prompt_message = 'Answer only with either "yes" or "no".'
prompt_message = 'Take a deep breath and think step by step, but make sure to end your response with a final answer either "yes" or "no".'
prompt_message = 'Take a deep breath and think step by step, Make sure to end your judgement based on the analysis and data provided with a final answer either "yes" or "no".'
prompt_message = 'Based on the available information and making reasonable estimations where needed, end your response with a final answer either "yes" or "no".'

In [68]:
for q in questions:
	story = stories[q['story_index']]
	print(story['index'])
	print(story['motivation']['introduction'])
	print(q['question'])
	print(prompt_message)
	print(q['answer'])

0
In light of Larisa Kuklina's emotional birthday celebration following her recent relay race, a closer look at her performance data could reveal intriguing patterns. By examining her race times, shooting accuracy, and results before and after her hand injury, we might uncover how such physical setbacks impact an athlete's overall performance. This exploration can shed light on the resilience and adaptability of athletes facing unexpected challenges.
For athletes with lower training intensity, is the impact of mental stress on performance greater than that of a hand injury?
Based on the available information and making reasonable estimations where needed, end your response with a final answer either "yes" or "no".
yes
0
In light of Larisa Kuklina's emotional birthday celebration following her recent relay race, a closer look at her performance data could reveal intriguing patterns. By examining her race times, shooting accuracy, and results before and after her hand injury, we might un